In [1]:
from typing import Dict
import json
from langgraph.graph import MessagesState
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
import uvicorn
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_together import ChatTogether
from pydantic import Field,BaseModel

In [4]:

llm = ChatGroq(
    api_key="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",
    model_name="gemma2-9b-it",
    temperature=0,
)


In [5]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Sample Portfolio Data
portfolio = pd.DataFrame({
    'stock': ['AAPL', 'TSLA', 'GOOGL', 'AMZN', 'MSFT'],
    'investment': [5000, 3000, 7000, 4000, 6000],
    'risk_tolerance': ['Low', 'High', 'Medium', 'High', 'Low']
})

# Market Data (Simulated)
market_trends = {
    'AAPL': "Apple is experiencing strong growth with new product launches.",
    'TSLA': "Tesla stock is volatile due to regulatory concerns.",
    'GOOGL': "Google's ad revenue is increasing, boosting its stock value.",
    'AMZN': "Amazon faces supply chain issues, impacting profits.",
    'MSFT': "Microsoft's cloud computing division is driving revenue growth."
}

# LangChain LLM Setup
 # Replace with your LLM API key
prompt_template = PromptTemplate(
    input_variables=["stock", "investment", "risk", "trend"],
    template=(
        "You have invested ${investment} in {stock} with a {risk} risk tolerance. "
        "The current market trend for {stock} is: {trend}. "
        "Provide personalized investment insights based on this information."
    )
)

def generate_investment_insights(row):
    prompt = prompt_template.format(
        stock=row['stock'],
        investment=row['investment'],
        risk=row['risk_tolerance'],
        trend=market_trends[row['stock']]
    )
    return llm.invoke(prompt)

# Generate Insights for Each Stock
portfolio['investment_insight'] = portfolio.apply(generate_investment_insights, axis=1)

print(portfolio[['stock', 'investment_insight']])

   stock                                 investment_insight
0   AAPL  content="It sounds like you're in a bit of a t...
1   TSLA  content="##  TSLA Investment Insights: High Ri...
2  GOOGL  content="##  GOOGL Investment Insights: Riding...
3   AMZN  content="##  AMZN Investment Insights: High Ri...
4   MSFT  content="It sounds like you're looking for som...
